In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, ReLU

from tensorflow.keras import losses
from tensorflow.keras import optimizers

In [6]:
def mlp_sequential(num_classes, input_dim, opts):
    model = Sequential()
    model.add(Input((input_dim,)))
    for hidden_dim in opts.layer_sizes:
        model.add(Dense(hidden_dim))
        model.add(ReLU())
    model.add(Dense(num_classes))
    model.compile(loss=losses.CategoricalCrossentropy(from_logits=True),
                  optimizer=optimizers.SGD(
                      learning_rate=opts.learning_rate,
                      momentum=opts.momentum),
                  metrics=['accuracy'])
    return model

In [14]:
from tensorflow.keras.models import Model

def mlp_functional(num_classes, input_dim, opts):
    inputs = Input((input_dim,), name="Pixels")
    h = inputs
    for hidden_dim in opts.layer_sizes:
        h = Dense(hidden_dim)(h)
        h = ReLU()(h)
    logits = Dense(num_classes, name="Classification_head")(h)
    model = Model(inputs=inputs, outputs=logits)
    model.compile(loss=losses.CategoricalCrossentropy(from_logits=True),
                  optimizer=optimizers.SGD(
                      learning_rate=opts.learning_rate,
                      momentum=opts.momentum),
                  metrics=['accuracy'])
    return model

In [7]:
import yaml
from types import SimpleNamespace

opts = yaml.load(open('hyper1.yaml'), Loader=yaml.Loader)
opts = SimpleNamespace(**opts)


In [15]:
from data import get_keras_mlp_data
data = get_keras_mlp_data(opts.dataset)
#model = mlp_sequential(data['num_classes'], data['input_dim'], opts)
model = mlp_functional(data['num_classes'], data['input_dim'], opts)

In [16]:
print(model.summary())

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Pixels (InputLayer)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 700)            │       549,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_6 (ReLU)                  │ (None, 700)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 500)            │       350,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_7 (ReLU)                  │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 500)            │       250,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_8 (ReLU)                  │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Classification_head (Dense)     │ (None, 10)             │         5,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,155,510 (4.41 MB)

 Trainable params: 1,155,510 (4.41 MB)

 Non-trainable params: 0 (0.00 B)

None


In [17]:
model.fit(*data['train'],
          epochs=opts.num_epochs,
          batch_size=opts.batch_size,
          validation_data=data['test'])


Epoch 1/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.6809 - loss: 1.1793 - val_accuracy: 0.9213 - val_loss: 0.2745
Epoch 2/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.9256 - loss: 0.2570 - val_accuracy: 0.9402 - val_loss: 0.2104
Epoch 3/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.9424 - loss: 0.2035 - val_accuracy: 0.9509 - val_loss: 0.1664
Epoch 4/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.9544 - loss: 0.1582 - val_accuracy: 0.9570 - val_loss: 0.1416
Epoch 5/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.9606 - loss: 0.1340 - val_accuracy: 0.9631 - val_loss: 0.1247
Epoch 6/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.9671 - loss: 0.1160 - val_accuracy: 0.9655 - val_loss: 0.1152
Epoch 7/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.9720 - loss: 0.0941 - val_accuracy: 0.9671 - val_loss: 0.1043
Epoch 8/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.9761 - loss: 0.0850 - val_acc

In [18]:
loss_and_acc = model.evaluate(*data['test'],
                              batch_size=opts.batch_size,
                              verbose=0)


In [19]:
loss_and_acc

[0.07076174020767212, 0.9771999716758728]